In [14]:
import json
import csv
from collections import defaultdict

team_data_file = "/data/old-teams.json"
puzzle_data_file = "/data/puzzle-data.json"
new_team_data_file = "/data/teams.json"

In [61]:
team_count = 0
teams_with_no_hints = 0
teams_with_some_hints = 0
teams_with_timeouts = 0
total_hints_used = []

new_teams = []
puzzle_data = defaultdict(lambda: {
    "timeouts": 0,
    "attempts": 0,
    "hints_used": 0
})

with open(team_data_file, 'r', encoding='utf-8') as f:
    data = json.load(f)
    
    for team in data:
        team_count += 1
        
        team_hints_used = 0
        team_timeouts = 0
        team_attempts = 0
        
        for puzzle_attempt in team["puzzles"]:
            if puzzle_attempt["start"] == None or puzzle_attempt["name"] == "Meta Puzzle":
                continue

            timeout = 1 if puzzle_attempt.get("timedOut", False) else 0
            puzzle_data[puzzle_attempt["name"]]["attempts"] += 1
            puzzle_data[puzzle_attempt["name"]]["timeouts"] += timeout
            puzzle_data[puzzle_attempt["name"]]["hints_used"] += puzzle_attempt["hintsTaken"]
            
            team_attempts += 1
            team_hints_used += puzzle_attempt["hintsTaken"]
            team_timeouts += timeout
        
        team_data = {
            "hints_used": team_hints_used,
            "timeouts": team_timeouts,
            "size": len(team.get("members")),
            "final_score": team.get('finalScore'),
            "division": team.get('division'),
        }
        
        if team_attempts == 4:
            new_teams.append(team_data)

with open(new_team_data_file, 'w', encoding='utf-8') as f:
    json.dump(new_teams, f)
    
with open(puzzle_data_file, 'w', encoding='utf-8') as f:
    json.dump(puzzle_data, f)